In [ ]:
import pandas as pd
import numpy as np
from Utils import extract_symbol_timeframe

# === EMA / SMA ===
def ema(arr, window):
    alpha = 2 / (window + 1)
    result = np.empty_like(arr)
    result[:] = np.nan
    result[0] = arr[0]
    for i in range(1, len(arr)):
        result[i] = alpha * arr[i] + (1 - alpha) * result[i - 1]
    return result

def sma(arr, window):
    return pd.Series(arr).rolling(window=window).mean().to_numpy()

# === Zero Lag MACD Enhanced ===
def compute_zero_lag_macd(close, fast_len, slow_len, signal_len, macd_ema_len, use_ema=True, use_old_algo=False):
    # Fast line
    ma1 = ema(close, fast_len) if use_ema else sma(close, fast_len)
    ma2 = ema(ma1, fast_len) if use_ema else sma(ma1, fast_len)
    zero_fast = 2 * ma1 - ma2

    # Slow line
    mas1 = ema(close, slow_len) if use_ema else sma(close, slow_len)
    mas2 = ema(mas1, slow_len) if use_ema else sma(mas1, slow_len)
    zero_slow = 2 * mas1 - mas2

    # MACD line
    zero_lag_macd = zero_fast - zero_slow

    # Signal
    if use_old_algo:
        signal = sma(zero_lag_macd, signal_len)
    else:
        emasig1 = ema(zero_lag_macd, signal_len)
        emasig2 = ema(emasig1, signal_len)
        signal = 2 * emasig1 - emasig2

    return zero_lag_macd, signal

# === Сделки ===
def extract_trades(df, macd, signal, tp=0.5, stop_loss_pct=99):
    macd_shift = np.roll(macd, 1)
    signal_shift = np.roll(signal, 1)

    # Вход по пересечению снизу вверх и ниже нуля
    cross_up = (macd_shift < signal_shift) & (macd > signal) & (macd < 0)
    cross_up[0] = False

    close = df["close"].to_numpy()
    high = df["high"].to_numpy()
    low = df["low"].to_numpy()
    open_time = df["open_time"].to_numpy()

    trades = []
    in_trade = False
    last_exit = 0

    for i in np.where(cross_up)[0]:
        if in_trade or i <= last_exit:
            continue

        entry_price = close[i]
        tp_price = entry_price * (1 + tp / 100)
        sl_price = entry_price * (1 - stop_loss_pct / 100)
        entry_time = open_time[i]

        stop_triggered = False
        stop_time = pd.NaT

        for j in range(i + 1, len(close)):
            if high[j] >= tp_price:
                exit_price = tp_price
                exit_time = open_time[j]
                break
            elif low[j] <= sl_price:
                exit_price = sl_price
                exit_time = open_time[j]
                stop_triggered = True
                stop_time = open_time[j]
                break
        else:
            break

        duration = int((exit_time - entry_time).total_seconds() // 60)

        trades.append({
            "entry_time": entry_time,
            "entry_price": entry_price,
            "macd": macd[i],
            "macd_signal": signal[i],
            "exit_time": exit_time,
            "exit_price": exit_price,
            "duration_min": duration,
            "stop_triggered": stop_triggered,
            "stop_time": stop_time
        })

        last_exit = j
        in_trade = False

    return pd.DataFrame(trades)

# === Загрузка и запуск ===
path = " " # Format filename = {symbol}USDT_timeframe_data.csv, example: "BTCUSDT_1h_from_1_Jan_2020.csv"
df = pd.read_csv(path)

symbol, timeframe = extract_symbol_timeframe(path)
indicator_name = "ZeroLag_MACD"

df["open_time"] = pd.to_datetime(df["open_time"], utc=True)
df.dropna(inplace=True)

# === Расчёт ===
macd, signal = compute_zero_lag_macd(
    close=df["close"].to_numpy(dtype=np.float64),
    fast_len=12,
    slow_len=26,
    signal_len=9,
    macd_ema_len=9,
    use_ema=True,
    use_old_algo=False
)

df = df.iloc[len(df) - len(macd):].copy()
df["macd"] = macd
df["macd_signal"] = signal

deals_df = extract_trades(df, macd, signal, tp=0.5, stop_loss_pct=99)
print("✅ Сделок найдено:", len(deals_df))
print(f"Криптовалюта: {symbol}\nТаймфрейм: {timeframe}\nСтратегия: {indicator_name}")
deals_df.tail()
